# Solving Navier-Stokes Numerically

Lecture 7.1  
Stephen Neethling

## Table of Contents
```{contents}
```

## Outline
- Discuss some of the relevant features of the Navier-Stokes equation
- Derive and discuss the fully implicit solution to the NS equation
- Discuss a semi-implicit method for solving the equation (Pressure projection method)
- Implement the pressure projection method
- Demonstrate and discuss the solution of some example problems

## Learning Objectives

- Derive an implicit discretisation for the NS equation
- Derive a pressure projection discretisation for solving the NS equation
- Derive appropriate boundary conditions for the problem
- Learn how to implement a solver for the Pressure Poisson problem
- Learn how to implement an explicit solver for the momentum
equation, including the appropriate use of up-winding
- Combine the solvers into a single Navier-Stokes solver capable of
solving a range of fluid flow problems

## The Navier-Stokes Equation

- Momentum balance:

$$ \rho \frac{D\mathbf{u}}{Dt} = - \nabla P + \nabla \cdot \mathbf{\tau} + \rho \mathbf{g} $$  

*Material derivative:* $\frac{D\mathbf{u}}{Dt} = \frac{\partial \mathbf{u}}{\partial t} + \mathbf{u} \cdot \nabla \mathbf{u}$  
*or equivalently:* $\frac{D\mathbf{u}}{Dt} = \frac{\partial \mathbf{u}}{\partial t} + \nabla \cdot \mathbf{u} \mathbf{u} $  

- Mass balance (continuity equation):

$$ \frac{\partial \rho}{ \partial t} + \nabla \cdot ( \rho \mathbf{u} ) = 0 $$

### Assumptions for this Lecture

- We will only be considering incompressible flow – constant density: $\nabla \cdot \mathbf{u} = 0$  
- We will start by considering Newtonian Flow:  
Shear stress is proportional to strain rate  

$$ \mathbf{\tau} = 2 \mu \mathbf{S} $$  

$$ \mathbf{S} = \frac{1}{2} (\nabla \mathbf{u} + \nabla \mathbf{u}^T)$$

- This assumes viscosity is isotropic and also ignores the bulk viscosity, which
is associated with changes in the volume/density of the fluid (we are
making an incompressible assumption here).  
Bulk viscosity can be important in, for instance shocks

### Incompressible Newtonian Fluid

- In an incompressible Newtonian fluid the gradient of viscous stress
can be further simplified if the viscosity is constant and the system is
incompressible:

$$ \nabla \cdot \mathbf{\tau} = \mu \nabla \cdot ( \nabla \mathbf{u} + \nabla \mathbf{u}^T ) = \mu \nabla^2 \mathbf{u} $$  

- Note that the incompressible assumption implies that:

$$ \nabla \cdot \mathbf{u} = 0 $$

#### Side note: Dynamic and Kinematic Viscosity

Be careful when talking about viscosity as to what is meant  

Dynamic viscosity (sometimes called absolute viscosity)
- 𝜇 (units: Pa.s)  
- Proportionality between shear stress and strain rate  

Kinematic viscosity
- $ \nu = \frac{\mu}{\rho}$ (units $m^2/s$)
- Equivalent to a diffusivity for momentum


### Incompressible Navier-Stokes Equation

In 3D this is 4 PDEs with 4 dependent variables (3 velocities and pressure) –
therefore solvable  

Momentum balance: $\rho \frac{\partial \mathbf{u}}{\partial t} + \rho \mathbf{u} \cdot \nabla \mathbf{u} = - \nabla P + \mu \nabla^2 \mathbf{u} + \rho \mathbf{g}$  

Incompressible continuity: $\nabla \cdot \mathbf{u} = 0$

Complexity:
- Non-linear  
    - The momentum advection term is non-lnear  
    - Additional non-linearity can occur if the rheology is non-Newtonian or if the system is compressible
- Coupled PDEs

At first glance the momentum equations seem easy to discretise
- If it was not for the pressure you could simply integrate them forward
in time
- The problem is that such an integration would not result in velocities
that obey the continuity equation
- What we need to do is integrate the momentum equations forward in
time, but have a pressure field that evolves to ensure that continuity
is obeyed


#### A couple of notes
Representing the data
- In the subsequent slides I will quite often represent the data as a vector:
     - E.g. the velocities at time step 𝑛 will be represented as $\mathbf{u^n}$
     - Within this vector will be stored each velocity component at every discrete point within the system
     - It is always possible to store higher dimensional data as 1D vector
     
Gravity
- In the subsequent derivations I am going to ignore the gravity term
- For an incompressible single phase system the gravity term can always be combined into the pressure term
     - This is not true in either a multiphase system or a compressible system where gravity would need to be explicitly
considered
- If 𝑃 is the actual pressure, then the gravity term can be eliminated by using a new pressure $ P^* $ that includes the hydrostatic contribution:  

$$ P^* = P - \rho \mathbf{x \cdot g}$$

- Where $\mathbf{x}$ is the location (i.e. $\nabla (\mathbf{x \cdot g}) = \mathbf{g}$

## Method 1: Implicit Matrix Solution for all Variables

- One way to achieve continuity is to simultaneously solve for the velocity and pressure
values
- Using a Crank-Nicolson implicit scheme (2nd order accurate in time)
    - Crank-Nicolson carries out the spatial derivatives using an average of the current and future time step
    
$$ \frac{\mathbf{u}^{n+1} - \mathbf{u}^n}{\Delta t} + \mathbf{A(\tilde{u})} + \frac{1}{2} (\mathbf{u}^{n+1} + \mathbf{u}^n) - \mathbf{K} \frac{1}{2} (\mathbf{u}^{n+1} - \mathbf{u}^n) + \mathbf{C P^{n+1}} = \mathbf{0} $$  

$$ \mathbf{C}^T \mathbf{u}^{n+1} = 0 $$  

- $\mathbf{K, C}$ and $\mathbf{C^T}$ are the matrices for the  discretisation of the viscosity (Laplacian), gradient operator and divergence respectively  
- $\mathbf{A(\tilde{u})}$ is the matrix containing the discretisation for the momentum advection term.  
    - Note that the matrix for the momentum advection term depends on the velocity. This is what make the problem non-linear
    
- The equations on the previous slide can be rearranged into a single matrix where
the velocities and pressures at the next time step are a function of the current
velocities:  

$$ \left( \begin{array} \\
\frac{1}{\Delta t} \mathbf{I} + \frac{1}{2} (\mathbf{A(\tilde{u}) - K}) & \mathbf{C} \\
\mathbf{C^T} & \mathbf{0} \\
\end{array} \right)
\left( \begin{array} \\
\mathbf{u^{n+1}} \\
\mathbf{P^{n+1}} \\
\end{array} \right) = 
\left( \begin{array} \\
\left( \frac{1}{\Delta t} \mathbf{I} + \frac{1}{2} (\mathbf{A(\tilde{u}) - K}) \right) \mathbf{u}^n \\  
\mathbf{0} \\
\end{array} \right)
$$

-  The next decision is the choice use for the velocity in the momentum advection term
    - The easiest, but least accurate is to use the velocity from the current time step, $\mathbf{u}^n$
    - More accurate would be to use the same average as used in the rest of the Crank-Nicolson scheme, $\frac{1}{2}(\mathbf{u}^{n+1} + \mathbf{u}^n)$. This is more accurate, by the problem is that it requires iteration and multiple assembly and solves of the matrix
- The big problem with this method is that it is computationally expensive because the matrix to be solved is large and difficult to solve because





## Method 2: Split the Pressure and Velocity Solves

- Firstly, we need an equation for the pressure solve
- We can achieve this by taking the divergence of the momentum equations:  

$$ \nabla \cdot \left( \frac{\partial \mathbf{u}}{\partial t} + \mathbf{u} + \nabla \mathbf{u} \right) = \nabla \cdot \left( - \frac{1}{\rho} \nabla P + \nu \nabla^2 \mathbf{u} \right)$$

- This can be simplified using the fact that the divergence of the Laplacian of the velocity and the divergence of the rate of change of velocity are both commutable and therefore zero given continuity:

$$ \nabla \cdot (\nabla^2 \mathbf{u}) = \nabla^2(\nabla \cdot \mathbf{u}) = 0$$  

$$ \nabla \cdot \left( \frac{\partial \mathbf{u}}{\partial t} \right) = \frac{\partial}{\partial t} ( \nabla \cdot \mathbf{u} ) = 0$$

## Pressure Poisson Equation

- This results in the following equation:

$$ \nabla^2 P = - \rho \nabla \cdot ( \mathbf{u} \cdot \nabla \mathbf{u}) $$  

- This is known as the pressure Poisson equation. It is very similar to the Laplace equation we solved in the previous lecture, except that it has a non-zero RHS
    - This equation is therefore quite easy to solve and generally well behaved in terms of its convergence
- In 2D this can be further simplified and expanded to:

$$ \frac{\partial^2 P}{\partial x^2} + \frac{\partial^2 P}{\partial y^2} = - \rho \left( \left( \frac{\partial u}{\partial x} \right)^2 + 2 \frac{\partial u}{\partial y} \frac{\partial v}{\partial x} + \left(\frac{\partial v}{\partial y} \right)^2\right)$$


### Finite Difference approximation of Pressure Poisson Equation

- We can do a finite difference approximation of the this equation
    - I will assume that $\Delta x = \Delta y$
    
$$ P_{i+1,j} + P_{i-1,j} + P_{i,j+1} + P_{i,j-1} - 4 P_{i,j} $$  

$$ = -\rho \left( (u_{i+1,j} - u_{i-1,j} )^2 + 2(u_{i,j+1} - u_{i,j-1}) (v_{i+1,j} - v_{i+1,j} ) + (v_{i,j+1} - v_{i,j-1} )^2 \right) $$  

- We could now simply use an explicit calculation of the new velocities using a discretisation of the momentum equation together with the current velocity and pressure equations and then use the new velocities to calculate the new pressures
- Would work for a while, but numerical errors will mean that continuity violations will grow with time


## Pressure Projection

- In this method we will calculate an intermediate velocity and then use this velocity to calculate the pressure gradient required to make the velocity at the next step divergence free
- Start by calculating a velocity $ \mathbf{u^* } $ that ignores the contribution from the pressure gradient (note that this is an explicit forward Euler approximation):

$$ \frac{ \mathbf{u^* } - \mathbf{u^n} }{\Delta t} + \mathbf{A}( \mathbf{ \tilde{u} } )( \mathbf{u}^n) - \mathbf{K} ( \mathbf{u}^n ) = \mathbf{0} $$  

- From the full equation we therefore know that:

$$ \frac{\mathbf{u}^{n+1} - \mathbf{u}^*}{\Delta t} = \frac{-1}{\rho} \nabla \mathbf{P}^{n+1} $$  

$$ \mathbf{u}^{n+1} = \mathbf{u}^* - \frac{\Delta t}{\rho} \nabla \mathbf{P}^{n+1} $$  

- If we take the divergence of both sides:

$$ \nabla \cdot \mathbf{u}^{n+1} = \nabla \cdot  \mathbf{u}^* - \frac{\Delta t}{\rho} \nabla^2 \mathbf{P}^{n+1} $$  

- What we are trying to enforce is that the final velocity is divergence free $( \nabla \cdot \mathbf{u}^{n+1} = 0 )$  

$$ \nabla^2 \mathbf{P}^{n+1} = \frac{\rho}{\Delta t} \nabla \cdot \mathbf{u}^* $$  

We therefore have the following calculation sequence:
- Calculate the intermediate velocity explicitly:

$$ \mathbf{u}^* = \mathbf{u}^n - \Delta t \mathbf{A} (\mathbf{u}^n)(\mathbf{u}^n) + \Delta t \mathbf{K} (\mathbf{u}^n) = \mathbf{0} $$

- Calculate the pressure implicitly using the following pressure Poisson equation (remember to impose any pressure boundary conditions):

$$ \nabla^2 \mathbf{P}^{n+1} = \frac{\rho}{\Delta t} \nabla \cdot \mathbf{u}^* $$

- Calculate the velocity at the new time step (also remember to impose any velocity boundary conditions):

$$ \mathbf{u}^{n+1} = \mathbf{u}^* - \frac{\Delta t}{\rho} \mathbf{P}^{n+1} $$

### Pressure Projection- Finite Difference (2D)

- Viscous term:

$$ \large \mathbf{K}(\mathbf{u}^n) = v \left( \begin{array} \\
\frac{u^n_{i+1,j} + u^n_{i-1,j} - 2 u^n_{i,j}}{\Delta x^2} + \frac{u^n_{i,j+1} + u^n_{i,j-1} - 2 u^n_{i,j}}{\Delta y^2} \\
\frac{v^n_{i+1,j} + v^n_{i-1,j} - 2 v^n_{i,j}}{\Delta x^2} + \frac{v^n_{i,j+1} + v^n_{i,j-1} - 2 v^n_{i,j}}{\Delta y^2} \\
\end{array} \right)
$$

- Momentum advection term:
    - Note that we need to use upwind discretisations of velocity to be stable at high Reynolds numbers
    
$$\large \mathbf{A} ( \mathbf{u}^n )( \mathbf{u}^n ) = \left( \begin{array} \\
\left( u^n_{i,j} 
\begin{cases} 
    \frac{u^n_{i,j} - u^n_{i-1,j}}{\Delta x} & \text{if }~~ u^n_{i,j} > 0 \\
    \frac{u^n_{i+1,j} - u^n_{i,j}}{\Delta x} & \text{if }~~ u^n_{i,j} < 0 \\
\end{cases} \right) + 
\left( v^n_{i,j} 
\begin{cases} 
    \frac{u^n_{i,j} - u^n_{i,j-1}}{\Delta y} & \text{if }~~ v^n_{i,j} > 0 \\
    \frac{u^n_{i,j+1} - u^n_{i,j}}{\Delta y} & \text{if }~~ v^n_{i,j} < 0 \\
\end{cases} \right) \\
\left( u^n_{i,j} 
\begin{cases} 
    \frac{u^n_{i,j} - v^n_{i-1,j}}{\Delta x} & \text{if }~~ u^n_{i,j} > 0 \\
    \frac{v^n_{i+1,j} - v^n_{i,j}}{\Delta x} & \text{if }~~ u^n_{i,j} < 0 \\
\end{cases} \right) + 
\left( v^n_{i,j} 
\begin{cases} 
    \frac{v^n_{i,j} - v^n_{i,j-1}}{\Delta y} & \text{if }~~ v^n_{i,j} > 0 \\
    \frac{v^n_{i,j+1} - v^n_{i,j}}{\Delta y} & \text{if }~~ v^n_{i,j} < 0 \\
\end{cases} \right) \\
\end{array} \right)
$$

- Laplacian of the Pressure:

$$ \nabla^2 \mathbf{P}^{n+1} = \frac{P^{n+1}_{i+1,j} + P^{n+1}_{i-1,j} - 2 P^{n+1}_{i,j}}{\Delta x^2} + \frac{P^{n+1}_{i,j+1} + P^{n+1}_{i,j-1} - 2 P^{n+1}_{i,j}}{\Delta y^2} $$

- Gradient of the Pressure:

$$ \large \nabla \mathbf{P}^{n+1} = 
\left( \begin{array} \\
\frac{P^{n+1}_{i+1,j} - P^{n+1}_{i-1,j}}{2 \Delta x} \\
\frac{P^{n+1}_{i,j+1} - P^{n+1}_{i,j-1}}{2 \Delta y} \\
\end{array} \right)
$$

- Divergence of the intermediate velocity:
- Centeral difference  
    
$$\large \nabla \cdot \mathbf{u}^* = \frac{u^*_{i+1,j} - u^*_{i-1,j}}{2 \Delta x} + \frac{v^*_{i,j+1} - v^*_{i,j-1}}{2 \Delta y}$$

- Upwind

$$\large \nabla \cdot \mathbf{u}^* = 
\begin{cases}
\frac{u^*_{i,j} - u^*_{i-1,j}}{\Delta x} & \text{if }~~ u^*_{i,j} > 0 \\
\frac{u^*_{i+1,j} - u^*_{i,j}}{\Delta x} & \text{if }~~ u^*_{i,j} < 0 \\
\end{cases} +
\begin{cases}
\frac{v^*_{i,j} - v^*_{i,j-1}}{\Delta y} & \text{if }~~ v^*_{i,j} > 0 \\
\frac{v^*_{i,j+1} - v^*_{i,j}}{\Delta y} & \text{if }~~ v^*_{i,j} < 0 \\
\end{cases}
$$

- Central difference is more accurate, but can cause problems especially at very high Reynolds numbers
- Upwinding is less accurate, but is more stable
- I will generally use central difference approximations for the RHS of the pressure Poisson equation


## Solid Boundaries in Fluid Dynamics

**No Slip Boundary Condition**
- A useful assumption for most solid boundaries
    - Together with no flux through the boundary this implies a zero velocity vector at the boundary
- Not completely true at the very smallest scales
    - Some molecular slip may need to be included for micro or nano scale fluid flows
    - Good approximation for macroscopic flows
- We will be using no slip conditions in all the examples in this lecture

**Velocity Boundary Layers**
- Some flows will have velocity boundary layers against solid walls
     - No slip at the wall, but rapid increase towards free stream velocity away from the wall
     - Boundary layers often thin compared to the scale of the system
- Especially true for turbulent flows
     - Can also occur in entry regions for laminar flows
     - More on turbulent flow in the next lecture
     
**Modelling Systems with Boundary Layers**
- Could simply have enough resolution near the wall to resolve the wall
     - Can be computationally very expensive especially in large scale simulations
- Simplest approximation is that there is full slip at the boundary
     - Not a bad approximation in very turbulent flows, but implies that there are no shear stresses on the wall
- Can use a sub-resolution model to impose a stress at the boundary that is a function of the velocity at the boundary
     - More complex models will include predictions for the velocity profile in this boundary region and a more complex inter-relationship between the stress at the wall and the velocity profile in the resolved near wall region

## Pressure Condition at No-Slip Boundary

- To solve the system we need boundary conditions for the pressure and the velocity
- Lets consider a vertical wall and the momentum balance in the direction normal to the wall:

$$\require{cancel} \cancel{{\frac{\partial u}{\partial t}}} + \cancel{u{\frac{\partial u}{\partial x}}} + \cancel{v{\frac{\partial u}{\partial y}}} + \cancel{w{\frac{\partial u}{\partial z}}} = - \frac{1}{\rho} \frac{\partial P}{\partial x} + \frac{\mu}{\rho} \left( \frac{\partial^2 u}{\partial x^2} + \cancel{{\frac{\partial^2 u}{\partial y^2}}} + \cancel{{\frac{\partial^2 u}{\partial z^2}}} \right)$$  

$$ \frac{\partial P}{\partial x} = \mu {\frac{\partial^2 u}{\partial x^2}} $$

- This says that the total normal stress at the boundary should be zero. For no slip boundaries the shear stress component into the boundary will be small (or zero for fully developed flows)
- The most commonly used pressure boundaries for these walls is thus:

$$ \large \frac{\partial P}{\partial n} = 0 $$

## The Code (with explicit loops)
- I am firstly going to show the code with explicit loops and if statements
     - Hopefully make what is being done easier to understand
- This is exactly how I would code this in a compiled language like C/C++
- Unfortunately Python is an interpreted language and very inefficient
in its implementation of for loops and conditions
    - Vectorised portions of the code are executed using libraries written in C++
and so are fast

### NOT SURE HOW TO PUT C++ IN YET


## Test Example

- Going to solve the code for the specific case of a lid driven cavity
- No inflow or outflow
- Top boundary has an applied velocity of 1
     - No slip w.r.t. this movement
- All other boundaries are stationary with no slip
- Box of size 1x1

$$ \large \rho = 1, \mu = v = 0.1 $$

- This gives a Reynold’s number based on the lid velocity of 10


### Note on the Pressure Calculation 
- In problems such as this where all the pressure boundaries are Neumann
boundaries (gradient boundaries) there is a potential issue with the
pressure calculations:
-  Flow only depends on the gradient of the pressure and therefore adding a
constant to all the pressure values does not change the solution
    - System is under-specified by one degree of freedom
    - A naïve iteration can therefore drift and never converge
- A solution to this problem is to set pressure at one point in the system
    - For more direct matrix solution methods for systems like this the removal of what is
known as the nullspace is required


### More C++

### Solution for Re = 10

